# Summarizing Multiple PDFs

In [ ]:
# !pip install langchain
# !pip install openai
# !pip install chromadb

In [1]:
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import PyPDFLoader
from langchain import OpenAI, PromptTemplate
import glob

In [2]:
llm = OpenAI(temperature=0.2)
def summarize_pdfs_from_folder(pdfs_folder):
    summaries = []
    for pdf_file in glob.glob(pdfs_folder + "/*.pdf"):
        loader = PyPDFLoader(pdf_file)
        docs = loader.load_and_split()
        chain = load_summarize_chain(llm, chain_type="map_reduce")
        summary = chain.run(docs)
        print("Summary for: ", pdf_file)
        print(summary)
        print("\n")
        summaries.append(summary)
    
    return summaries

In [3]:
def custom_summary(pdf_folder, custom_prompt):
    summaries = []
    for pdf_file in glob.glob(pdf_folder + "/*.pdf"):
        loader = PyPDFLoader(pdf_file)
        docs = loader.load_and_split()
        prompt_template = custom_prompt + """

        {text}

        SUMMARY:"""
        PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])
        chain = load_summarize_chain(llm, chain_type="map_reduce", 
                                    map_prompt=PROMPT, combine_prompt=PROMPT)
        summary_output = chain({"input_documents": docs},return_only_outputs=True)["output_text"]
        summaries.append(summary_output)
        
    return summaries

In [4]:
summaries = summarize_pdfs_from_folder("./pdfs")

Summary for:  ./pdfs/llm_planning_paper.pdf
 This paper introduces LLM+P, a framework that combines the strengths of large language models (LLMs) and classical planners to solve long-horizon planning problems. LLM+P takes a natural language description of a problem, converts it into a file written in the planning domain definition language (PDDL), uses classical planners to find a solution, and then translates the solution back into natural language. Experiments on a set of benchmark problems show that LLM+P is able to provide optimal solutions, while LLMs fail to provide even feasible plans. The paper also discusses the limitation of not recognizing when a prompt should be processed by LLM+P, and provides examples of failures of LLM-AS-P and LLM+P.


Summary for:  ./pdfs/prompt_vision_language_models_paper.pdf


This paper proposes Context Optimization (CoOp), a method for adapting pre-trained vision-language models for downstream image recognition tasks. CoOp models a prompt's contex

In [30]:
# CUSTOM_PROMPT = "Write a concise summary of the following paper with this structure: Problem being solved; Approach; Main results; Main Discussion Points"
# custom_summaries = custom_summary("./pdfs", custom_prompt=CUSTOM_PROMPT)
# # Save all summaries into one .txt file
# with open("custom_summaries.txt", "w") as f:
#     for summary in custom_summaries:
#         f.write(summary + "\n"*3)

In [21]:
# Save all summaries into one .txt file
with open("summaries-xxf.txt", "w") as f:
    for summary in summaries:
        f.write(summary + "\n"*3)

# Querying Multiple PDFS

In [22]:
# .py
from langchain.indexes import VectorstoreIndexCreator
from langchain.document_loaders import PyPDFDirectoryLoader

In [23]:
# Python!
loader = PyPDFDirectoryLoader("./pdfs/")

docs = loader.load()

# Create the vector store index
index = VectorstoreIndexCreator().from_loaders([loader])

Using embedded DuckDB without persistence: data will be transient


In [24]:
query = "What is the core idea behind the CoOP (context optimization) paper?"

index.query(query)

" The core idea behind the CoOP paper is to model a prompt's context words with learnable vectors while keeping the entire pre-trained parameters fixed, in order to adapt CLIP-like vision-language models for downstream image recognition tasks."

In [25]:
query = "What is the central idea that can allow for scaling transformers to 1 million tokens?"

index.query(query)

' The central idea is to use the Recurrent Memory Transformer (RMT) architecture to extend the context length of BERT, allowing it to store and process both local and global information across up to 2 million tokens.'

In [26]:
query = "According to the LLM+P paper, how do you empower large language models with optimal planning profficiency?"

index.query(query)

' LLM+P takes in a natural language description of a planning problem, then returns a correct (or optimal) plan for solving that problem in natural language. LLM+P does so by first converting the language description into a file written in the planning domain definition language (PDDL), then leveraging classical planners to quickly find a solution, and then translating the found solution back into natural language.'